# 0. Install Dependencies and Bring in Data

In [55]:
# !pip install tensorflow tensorflow-gpu pandas matplotlib sklearn
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [56]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [57]:
import pandas as pd
df=pd.read_csv("/content/gdrive/My Drive/dl/train.csv")





In [58]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [59]:
df.iloc[4]['comment_text']

"You, sir, are my hero. Any chance you remember what page that's on?"

In [60]:
df[df['threat']==1] .head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
79,003217c3eb469ba9,Hi! I am back again!\nLast warning!\nStop undo...,1,0,0,1,0,0
176,006b94add72ed61c,I think that your a Fagget get a oife and burn...,1,0,1,1,1,1
600,0199d6af27b715f3,I'm also a sock puppet of this account...SUPRI...,1,0,0,1,0,0
802,02230885017a50c5,"Fuck you, Smith. Please have me notified when ...",1,0,1,1,1,0
1017,02c6e41e4b317ac3,WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'L...,1,1,1,1,1,1


In [61]:
df[df.columns[2:]].iloc[79]

toxic            1
severe_toxic     0
obscene          0
threat           1
insult           0
identity_hate    0
Name: 79, dtype: int64

# 1. Preprocess

In [62]:
# !pip list

In [63]:
from tensorflow.keras.layers import TextVectorization

In [64]:
TextVectorization??

In [65]:
print(df.columns)
df.columns[2:]

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')


Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [66]:
df[df.columns[2:]].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [67]:
X = df['comment_text']
y = df[df.columns[2:]].values #labels

In [68]:
MAX_FEATURES = 200000
# number of words in the vocab

In [69]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')
# //map every single word to an integer value

In [70]:
vectorizer.adapt(X.values)
# teach it vocabulary

In [71]:
# vectorizer.get_vocabulary()
# vectorizer.get_vocabulary??
vectorizer("hello world python")

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([ 288,  263, 9696, ...,    0,    0,    0])>

In [72]:
vectorized_text = vectorizer(X.values)

In [73]:
vectorized_text
# 159571 sentences  1800 maximum no.. of words
# if a sentence doesnt reach max it will fill it with zeroes

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [74]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [75]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

# 2. Create Sequential Model

In [76]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [77]:
model = Sequential()
# Create the embedding layer
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer
model.add(Dense(6, activation='sigmoid'))

In [78]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [79]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                16640     
 onal)                                                           
                                                                 
 dense_8 (Dense)             (None, 128)               8320      
                                                                 
 dense_9 (Dense)             (None, 256)               33024     
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 6)                 774       
                                                      

In [ ]:
history = model.fit(train, epochs=5, validation_data=val)

Epoch 1/5
2132/6981 [========>.....................] - ETA: 2:14:09 - loss: 0.0818

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

# 3. Make Predictions

In [ ]:
input_text = vectorizer('You freaking suck! I am going to hit you.')
input_text #this will fail
# we have to pass it as a sequence if values

In [ ]:
np.array(input_text)

In [ ]:
np.expand_dims(input_text,0)

In [ ]:
df.columns[2:]

In [ ]:
model.predict(np.expand_dims(input_text,0))

In [ ]:
res = model.predict(np.expand_dims(input_text,0))

In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [ ]:
(model.predict(batch_X) > 0.5).astype(int)
# multiple comments passsed

In [ ]:
res.shape

# 4. Evaluate Model

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    yhat = model.predict(X_true)

    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

# 5. Test

In [ ]:
# !pip install gradio jinja2
!pip install gradio==3.50

In [ ]:
import tensorflow as tf
# import gradio as gr

In [ ]:
import pickle
from keras.models import load_model

# Assuming you have a vectorizer named 'vectorizer'
# Save the vectorizer
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Save the model
model.save('toxicity.h5')


In [ ]:
model.save('toxicity.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = tf.keras.models.load_model('toxicity.h5')

In [ ]:
input_str = vectorizer('hey i freaken hate you!')

In [ ]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 2s 2s/step


In [ ]:
res

array([[3.2425989e-04, 1.6851237e-08, 2.2371982e-05, 7.4409286e-07,
        4.3811266e-05, 1.4199511e-06]], dtype=float32)

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text

In [ ]:
print( score_comment("i hate you black nigger") )


1/1 [==============================] - 0s 54ms/step
toxic: True
severe_toxic: False
obscene: True
threat: False
insult: True
identity_hate: False



In [ ]:
print( score_comment("you are awesome") )

1/1 [==============================] - 0s 228ms/step
toxic: False
severe_toxic: False
obscene: False
threat: False
insult: False
identity_hate: False



In [ ]:
def check(comment):
  flag=1
  input_str = vectorizer(comment)
  res = model.predict(np.expand_dims(input_str,0))
  for i in range(6):
    if res[0][i] >=0.5:
      print("The comment is inappropriate")
      flag=0
      break
  if(flag):
    print("The comment is fine")
check("die")

1/1 [==============================] - 0s 84ms/step
The comment is inappropriate


In [ ]:
check("hey i freaken hate you nigger!")
check("you are awesome")
check("wow it was helpful")
check("go kill yourself")

1/1 [==============================] - 0s 55ms/step
The comment is inappropriate
1/1 [==============================] - 0s 55ms/step
The comment is fine
1/1 [==============================] - 0s 54ms/step
The comment is fine
1/1 [==============================] - 0s 54ms/step
The comment is inappropriate


## Filtering comments

In [ ]:
l=["hey i freaken hate you nigger!",
   "you are awesome",
   "wow it was helpful" ,
   "go kill yourself",
   "you are ugly",
   "you are not good",
   "you are garbage",
   "thanks that was useful"]
l1=[]
for s in l:
  flag=1
  input_str = vectorizer(s)
  res = model.predict(np.expand_dims(input_str,0))
  for i in range(6):
    if res[0][i] >=0.5:
      flag=0
      break
  if(flag):
    l1.append(s)


1/1 [==============================] - 0s 54ms/step


In [ ]:
for i in l1:
  print(i)

you are awesome
wow it was helpful
you are not good
thanks that was useful
